In [1]:
!pip install --upgrade implicit

In [2]:
import os; os.environ['OPENBLAS_NUM_THREADS']='1'
import numpy as np
import pandas as pd
import implicit
from scipy.sparse import coo_matrix
from implicit.evaluation import mean_average_precision_at_k

In [3]:
base_path = 'C://Users//darli//Desktop//527//group project//h-and-m-personalized-fashion-recommendations//'
csv_train = f'{base_path}transactions_train.csv'
csv_sub = f'{base_path}sample_submission.csv'
csv_users = f'{base_path}customers.csv'
csv_items = f'{base_path}articles.csv'

df = pd.read_csv(csv_train, dtype={'article_id': str}, parse_dates=['t_dat'])
df_sub = pd.read_csv(csv_sub)
dfu = pd.read_csv(csv_users)
dfi = pd.read_csv(csv_items, dtype={'article_id': str})

In [4]:
print(df['t_dat'].shape)
print(df['t_dat'].max())
print(df['t_dat'].min())
df1 = df[df['sales_channel_id'] ==1]
print(df1.shape)

(31788324,)
2020-09-22 00:00:00
2018-09-20 00:00:00
(9408462, 5)


In [5]:
print(df.isnull().sum())

t_dat               0
customer_id         0
article_id          0
price               0
sales_channel_id    0
dtype: int64


In [6]:
# use the data after 2020-08-21 to find the best parameters (factors, regularization and interations)
df = df[df['t_dat'] > '2020-08-21']
df.shape

(1190911, 5)

In [7]:
print(df.columns)
print(dfu.columns)

Index(['t_dat', 'customer_id', 'article_id', 'price', 'sales_channel_id'], dtype='object')
Index(['customer_id', 'FN', 'Active', 'club_member_status',
       'fashion_news_frequency', 'age', 'postal_code'],
      dtype='object')


In [8]:
ALL_USERS = dfu['customer_id'].unique().tolist()
ALL_ITEMS = dfi['article_id'].unique().tolist()

user_ids = dict(list(enumerate(ALL_USERS)))
item_ids = dict(list(enumerate(ALL_ITEMS)))

# dict_items = user_ids.items()
# first_ten = list(dict_items)[:10]
# print('user_ids:\n',first_ten)

user_map = {u: uidx for uidx, u in user_ids.items()}
item_map = {i: iidx for iidx, i in item_ids.items()}
# print('user_id 1:\n',df['customer_id'])

df['user_id'] = df['customer_id'].map(user_map)
# print('user_id 2:\n',df['user_id'])
df['item_id'] = df['article_id'].map(item_map)

del dfu, dfi

In [9]:
row = df['user_id'].values
col = df['item_id'].values
data = np.ones(df.shape[0])  #if purchase, set as one
coo_train = coo_matrix((data, (row, col)), shape=(len(ALL_USERS), len(ALL_ITEMS)))
coo_train

<1371980x105542 sparse matrix of type '<class 'numpy.float64'>'
	with 1190911 stored elements in COOrdinate format>

In [10]:
model = implicit.als.AlternatingLeastSquares(factors=10, iterations=2)
model.fit(coo_train)

C:\Users\darli\anaconda3\lib\site-packages\implicit\utils.py:31: UserWarning: Intel MKL BLAS detected. Its highly recommend to set the environment variable 'export MKL_NUM_THREADS=1' to disable its internal multithreading
  warnings.warn(


  0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
def to_user_item_coo(df):
    row = df['user_id'].values
    col = df['item_id'].values
    data = np.ones(df.shape[0])
    coo = coo_matrix((data, (row, col)), shape=(len(ALL_USERS), len(ALL_ITEMS)))
    return coo


def split_data(df, validation_days=7):
    validation_cut = df['t_dat'].max() - pd.Timedelta(validation_days)

    df_train = df[df['t_dat'] < validation_cut]
    df_val = df[df['t_dat'] >= validation_cut]
    return df_train, df_val

def get_val_matrices(df, validation_days=7):
    df_train, df_val = split_data(df, validation_days=validation_days)
    coo_train = to_user_item_coo(df_train)
    coo_val = to_user_item_coo(df_val)

    csr_train = coo_train.tocsr()
    csr_val = coo_val.tocsr()
    
    return {'coo_train': coo_train,
            'csr_train': csr_train,
            'csr_val': csr_val          }


def validate(matrices, factors=200, iterations=20, regularization=0.01, show_progress=True):
    
    coo_train, csr_train, csr_val = matrices['coo_train'], matrices['csr_train'], matrices['csr_val']
    
    model = implicit.als.AlternatingLeastSquares(factors=factors, 
                                                 iterations=iterations, 
                                                 regularization=regularization, 
                                                 random_state=42)
    model.fit(coo_train, show_progress=show_progress)
    
    map12 = mean_average_precision_at_k(model, csr_train, csr_val, K=12, show_progress=show_progress, num_threads=4)
#     print(f"Factors: {factors} - Iterations: {iterations} - Regularization: {regularization} ==> MAP@12: {map12}")
    return map12

In [ ]:
matrices = get_val_matrices(df)

best_map12 = 0
for factors in [40, 50, 60, 100, 200, 500, 1000]:
    for iterations in [3, 12, 14, 15, 20]:
        for regularization in [0.01]:
            map12 = validate(matrices, factors, iterations, regularization, show_progress=False)
            if map12 > best_map12:
                best_map12 = map12
                best_params = {'factors': factors, 'iterations': iterations, 'regularization': regularization}
                print(f"The best MAP@12 is unpdated. : {best_params}")

days in preset max:  2020-09-22 00:00:00
days in preset min:  2020-08-22 00:00:00
The best MAP@12 is unpdated. : {'factors': 40, 'iterations': 3, 'regularization': 0.01}
The best MAP@12 is unpdated. : {'factors': 40, 'iterations': 12, 'regularization': 0.01}
The best MAP@12 is unpdated. : {'factors': 50, 'iterations': 12, 'regularization': 0.01}
The best MAP@12 is unpdated. : {'factors': 60, 'iterations': 12, 'regularization': 0.01}
The best MAP@12 is unpdated. : {'factors': 60, 'iterations': 14, 'regularization': 0.01}
The best MAP@12 is unpdated. : {'factors': 60, 'iterations': 20, 'regularization': 0.01}
The best MAP@12 is unpdated. : {'factors': 100, 'iterations': 12, 'regularization': 0.01}
The best MAP@12 is unpdated. : {'factors': 100, 'iterations': 14, 'regularization': 0.01}
The best MAP@12 is unpdated. : {'factors': 200, 'iterations': 3, 'regularization': 0.01}
The best MAP@12 is unpdated. : {'factors': 500, 'iterations': 3, 'regularization': 0.01}


In [ ]:
del matrices

In [ ]:
base_path = 'C://Users//darli//Desktop//527//group project//h-and-m-personalized-fashion-recommendations//'
csv_train = f'{base_path}transactions_train.csv'
csv_sub = f'{base_path}sample_submission.csv'
csv_users = f'{base_path}customers.csv'
csv_items = f'{base_path}articles.csv'

df = pd.read_csv(csv_train, dtype={'article_id': str}, parse_dates=['t_dat'])
df_sub = pd.read_csv(csv_sub)
dfu = pd.read_csv(csv_users)
dfi = pd.read_csv(csv_items, dtype={'article_id': str})

In [ ]:
ALL_USERS = dfu['customer_id'].unique().tolist()
ALL_ITEMS = dfi['article_id'].unique().tolist()

user_ids = dict(list(enumerate(ALL_USERS)))
item_ids = dict(list(enumerate(ALL_ITEMS)))

# dict_items = user_ids.items()
# first_ten = list(dict_items)[:10]
# print('user_ids:\n',first_ten)

user_map = {u: uidx for uidx, u in user_ids.items()}
item_map = {i: iidx for iidx, i in item_ids.items()}
# print('user_id 1:\n',df['customer_id'])

df['user_id'] = df['customer_id'].map(user_map)
# print('user_id 2:\n',df['user_id'])
df['item_id'] = df['article_id'].map(item_map)

del dfu, dfi

In [ ]:
coo_train = to_user_item_coo(df)
csr_train = coo_train.tocsr()

In [ ]:
def train(coo_train, factors=200, iterations=15, regularization=0.01, show_progress=True):
    model = implicit.als.AlternatingLeastSquares(factors=factors, 
                                                 iterations=iterations, 
                                                 regularization=regularization, 
                                                 random_state=42)
    model.fit(coo_train, show_progress=show_progress)
    return model

In [ ]:
best_params

In [ ]:
model = train(coo_train, **best_params)

In [ ]:
def submit(model, csr_train, submission_name="submissions.csv"):
    preds = []
    batch_size = 2000
    to_generate = np.arange(len(ALL_USERS))
    for startidx in range(0, len(to_generate), batch_size):
        batch = to_generate[startidx : startidx + batch_size]
        ids, scores = model.recommend(batch, csr_train[batch], N=12, filter_already_liked_items=False)
        for i, userid in enumerate(batch):
            customer_id = user_ids[userid]
            user_items = ids[i]
            article_ids = [item_ids[item_id] for item_id in user_items]
            preds.append((customer_id, ' '.join(article_ids)))

    df_preds = pd.DataFrame(preds, columns=['customer_id', 'prediction'])
    df_preds.to_csv(submission_name, index=False)
    
    display(df_preds.head())
    print(df_preds.shape)
    
    return df_preds

In [ ]:
df_preds = submit(model, csr_train)